In [1]:
from rdflib import Graph, URIRef, Namespace, Literal, XSD, RDF
from rdflib.plugins.stores.sparqlstore import SPARQLStore
from tqdm.autonotebook import tqdm
import pandas as pd
from copy import deepcopy
import numpy
from sklearn.metrics.pairwise import cosine_similarity
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import defaultdict
from rapidfuzz import process, fuzz, distance
import math

import torch
from transformers import BertTokenizer
from transformers import BertModel
from sklearn.metrics.pairwise import cosine_similarity

import concurrent.futures

import numpy as np
import matplotlib.pyplot as plt


/var/folders/6f/k2t20zbs4db5khp6mgdq_06m0000gn/T/ipykernel_60205/3407715408.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
GRAPH_1_INV_FUNC_PATH = ''
GRAPH_1 = ""
GRAPH_2_INV_FUNC_PATH = ''
#GRAPH_2 = "/Users/yamamotobikutorueiichi/codes/custom_paris/marvel.xml"
GRAPH_2 = "http://localhost:8890/sparql"
SUBJECT = 'subject'
PREDICATE = 'predicate'
INVERSE_FUNCTIONALITY = 'inverse_functionality'
INVERSABILITY = 'inversability'
MAX_LENGTH_FULL_MATCH = 10000
LABEL_PREDICATE = "http://www.w3.org/2000/01/rdf-schema#label"
FILE_FOLDER = ""
GRAPH_1_ABSTRACT = "http://dbkwik.webdatacommons.org/ontology/abstract"
GRAPH_2_ABSTRACT = "http://dbpedia.org/ontology/abstract"

In [3]:
graph_1_inv_func_df = pd.read_csv(GRAPH_1_INV_FUNC_PATH)

#graph_1_predicate_list = threshold_graph_1_inv_func_df[PREDICATE].tolist()

graph_1 = Graph()
graph_1.parse(GRAPH_1)

<Graph identifier=N7ec256f87ae74c9bbf9dcac1edc67f63 (<class 'rdflib.graph.Graph'>)>

In [4]:
graph_2_inv_func_df = pd.read_csv(GRAPH_2_INV_FUNC_PATH)

#graph_2_predicate_list = threshold_graph_2_inv_func_df[PREDICATE].tolist()

#graph_2 = Graph()
#graph_2.parse(GRAPH_2)
graph_2 = Graph("SPARQLStore")
graph_2.open(GRAPH_2)

In [5]:
def append_instances(entity_dict, graph, offset):
    query = """
            SELECT ?instance ?label
            WHERE {{
              ?instance <http://www.w3.org/2000/01/rdf-schema#label> ?label.
            }}
            ORDER BY ?instance
            LIMIT 10000
            OFFSET {offset}
            """.format(offset=offset)
    attempts = 0
    while attempts < 10:
        try:
            for res in graph.query(query):
                entity_dict.setdefault(str(res[1]), []).append(str(res[0]))
            return
        except Exception as e:
            print(e)
            attempts += 1

In [6]:
def append_instances_alt_label(entity_dict, graph, offset):
    query = """
            SELECT ?instance ?label
            WHERE {{
              ?instance <http://www.w3.org/2004/02/skos/core#altLabel> ?label.
            }}
            ORDER BY ?instance
            LIMIT 10000
            OFFSET {offset}
            """.format(offset=offset)
    attempts = 0
    while attempts < 10:
        try:
            for res in graph.query(query):
                entity_dict.setdefault(str(res[1]), []).append(str(res[0]))
            return
        except Exception as e:
            print(e)
            attempts += 1

In [7]:
def get_graph_entities(graph):
    # Create a SPARQL query to get the total number of labels
    query_count = """
    SELECT (COUNT(?instance) as ?count)
    WHERE {
          ?instance <http://www.w3.org/2000/01/rdf-schema#label> ?label.
    }
    """
    total_instances = 0
    for res in graph.query(query_count):
        total_instances = int(res[0])
    print(total_instances)
    entity_dict = dict()
    
    for offset in tqdm(range(0, total_instances, 10000)):
        append_instances(entity_dict, graph, offset)

    query_count = """
    SELECT (COUNT(?instance) as ?count)
    WHERE {
          ?instance <http://www.w3.org/2004/02/skos/core#altLabel> ?label.
    }
    """
    total_instances = 0
    for res in graph.query(query_count):
        total_instances = int(res[0])
    print(total_instances)
    
    for offset in tqdm(range(0, total_instances, 10000)):
        append_instances_alt_label(entity_dict, graph, offset)
        
    return entity_dict

In [8]:
graph_1_label_dict = get_graph_entities(graph_1)

49960


  0%|          | 0/5 [00:00<?, ?it/s]

65400


  0%|          | 0/7 [00:00<?, ?it/s]

In [10]:
query = """
SELECT distinct ?instance
WHERE {{
  ?instance <http://www.w3.org/2000/01/rdf-schema#label> ?label.
  filter(?label="{labelChange}"@en)
}}""".format(labelChange='Harry Potter')
print(query)

for res in graph_2.query(query):
    print(res)


SELECT distinct ?instance
WHERE {
  ?instance <http://www.w3.org/2000/01/rdf-schema#label> ?label.
  filter(?label="Harry Potter"@en)
}
(rdflib.term.URIRef('http://dbpedia.org/resource/Category:Harry_Potter'),)
(rdflib.term.URIRef('http://dbpedia.org/resource/Harry_Potter'),)


In [11]:
def get_instances_by_label(graph, label):
    query = """
    SELECT distinct ?instance
    WHERE {{
      ?instance <http://www.w3.org/2000/01/rdf-schema#label> ?label.
      filter(?label="{labelChange}"@en)
    }}""".format(labelChange=label)
    
    entity_list = list()
    attempts = 0
    while attempts < 10:
        try:
            for res in graph_2.query(query):
                entity_list.append(res[0])
            return entity_list
        except Exception as e:
            print(e)
            attempts += 1
    return None

In [13]:
matching_list = list()

for label in tqdm(graph_1_label_dict):
    graph_2_entities = get_instances_by_label(graph_2, label)
    
    if not graph_2_entities or len(graph_2_entities) == 0:
        continue
    
    for e1 in graph_1_label_dict[label]:
        for e2 in graph_2_entities:
            matching_list.append({
                'e1': e1,
                'e2': e2,
                'sim': 1.0
            })

  0%|          | 0/68946 [00:00<?, ?it/s]

You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQ

You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQL query
You did something wrong formulating either the URI or your SPARQ

In [14]:
nltk_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
def embed_long_sentence(long_sentence):
    sentences = nltk_tokenizer.tokenize(long_sentence)
    
    tokens = {'input_ids': [], 'attention_mask': []}

    max_token = 140
    for sentence in sentences:
        new_tokens = tokenizer.encode_plus(sentence, return_tensors="pt", max_length=max_token, padding='max_length')
        tokens['input_ids'].append(new_tokens['input_ids'][0][:max_token])
        tokens['attention_mask'].append(new_tokens['attention_mask'][0][:max_token])

    tokens['input_ids'] = torch.stack(tokens['input_ids'])
    tokens['attention_mask'] = torch.stack(tokens['attention_mask'])
    
    outputs = model(**tokens)
    
    embeddings = outputs.last_hidden_state
    
    attention_mask = tokens['attention_mask']
    
    mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()

    masked_embeddings = embeddings * mask
    
    summed = torch.sum(masked_embeddings, 1)

    summed_mask = torch.clamp(mask.sum(1), min=1e-9)

    mean_pooled = summed / summed_mask
    
    return mean_pooled.detach().numpy()